In [ ]:
!unzip '/content/titanic.zip'

Archive:  /content/titanic.zip
  inflating: gender_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC #svm for classifying problems
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, StackingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [ ]:
df_train = pd.read_csv('/content/train.csv')

In [ ]:
df_test = pd.read_csv('/content/test.csv')

In [ ]:
df_submition = pd.read_csv('/content/gender_submission.csv')

In [ ]:
df_submition.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [ ]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [ ]:
#removing non-relevant columns
df_train = df_train.drop(['Name','Ticket','Cabin'],axis=1)
df_test = df_test.drop(['Name','Ticket','Cabin'],axis=1)

#removing rows with null values from class Embarked 
df_train = df_train.dropna(subset=['Embarked'])

#normalizing columns 'Sex' and 'Embarked' 
df_train = pd.get_dummies(df_train, prefix=['Sex','Embarked'], columns=['Sex','Embarked'])
df_test = pd.get_dummies(df_test, prefix=['Sex','Embarked'], columns=['Sex','Embarked'])

In [ ]:
df_train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare',
       'Sex_female', 'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S'],
      dtype='object')

In [ ]:
#filling NaN values by the column mean

df_train['Age'].fillna(int(df_train['Age'].mean()), inplace=True)
df_test['Age'].fillna(int(df_test['Age'].mean()), inplace=True)
df_test['Fare'].fillna(int(df_test['Fare'].mean()), inplace=True)

In [ ]:
#splitting the data (x and y)
x = df_train.drop(['Survived'],axis=1)
y = df_train['Survived']

#doing the predictions

In [ ]:
#normalizing predictor variables
#the svm works with distance calculation, so it's important to make the data in the same scale

normalizer = MinMaxScaler(feature_range=(0,1))

x_norm = normalizer.fit_transform(x)

In [ ]:
#model


modelSVM = SVC(kernel='poly',degree=5,probability=True)

#other models (just for comparision)
knn = KNeighborsClassiir()

rf = RandomForestClassifier(n_estimators = 1000, max_depth=100, random_state=1)

In [ ]:
#train/test (SKFold)
skfold = StratifiedKFold(n_splits=5)

svm_result = cross_val_score(modelSVM,x_norm,y,cv = skfold,n_jobs=-1)

knn_result = cross_val_score(knn,x_norm,y,cv = skfold,n_jobs=-1)

rf_result = cross_val_score(rf,x_norm,y,cv = skfold,n_jobs=-1)

In [ ]:
print(svm_result.mean())
print(knn_result.mean())
print(rf_result.mean())

0.8177870881736812
0.7986986605725894
0.8054402336062972


#Realizando Ajuste de Parâmetros

In [ ]:
#defining values
kernel = ['linear','poly','rbf','sigmoid']
polinom_degree = np.array([1,2,3,4,5])
constant = np.array([0.8,0.9,0.95,1.0,1.05,1.1,1.2])
grid_values = {'C':constant, 'kernel':kernel, 'degree':polinom_degree}

#creating the model
model = SVC()

#testing
gridSVR = GridSearchCV(estimator = model, param_grid = grid_values, cv=5, n_jobs=-1)
gridSVR.fit(x_norm,y)

GridSearchCV(cv=5, estimator=SVC(), n_jobs=-1,
             param_grid={'C': array([0.8 , 0.9 , 0.95, 1.  , 1.05, 1.1 , 1.2 ]),
                         'degree': array([1, 2, 3, 4, 5]),
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']})

In [ ]:
#Printing best parameters
print(f'Best C: {gridSVR.best_estimator_.C}')
print(f'Best Kernel: {gridSVR.best_estimator_.kernel}')
print(f'Best Degree: {gridSVR.best_estimator_.degree}')
print(f'Acuraccy: {gridSVR.best_score_}')

Best C: 1.0
Best Kernel: poly
Best Degree: 5
Acuraccy: 0.8177870881736812


after the previous analysis, we can conclude that the svm model with this specific parameters it's the best model for the challenge

#finalizando com as previsões

In [ ]:
#splitting the dataset (train and test)
x_train,x_test,y_train,y_test = train_test_split(x_norm,y,test_size=0.3,random_state=100)

In [ ]:
#training:
modelSVM = modelSVM.fit(x_train,y_train)

In [ ]:
predict_y = modelSVM.predict(x_test)

In [ ]:
print(f"Acurácia: {accuracy_score(y_test,predict_y)}")

Acurácia: 0.8164794007490637


In [ ]:
df_test.head()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,892,3,34.5,0,0,7.8292,0,1,0,1,0
1,893,3,47.0,1,0,7.0000,1,0,0,0,1
2,894,2,62.0,0,0,9.6875,0,1,0,1,0
3,895,3,27.0,0,0,8.6625,0,1,0,0,1
4,896,3,22.0,1,1,12.2875,1,0,0,0,1


In [ ]:
x_test_2 = df_test

normalizer_x_test = MinMaxScaler(feature_range=(0,1))

x_norm_test = normalizer.fit_transform(x_test_2)

predict_y_2 = modelSVM.predict(x_norm_test)

In [ ]:
predict_y_2

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [ ]:
df_test['Survived'] = predict_y_2
df_test.head(10)

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Survived
0,892,3,34.5,0,0,7.8292,0,1,0,1,0,0
1,893,3,47.0,1,0,7.0000,1,0,0,0,1,0
2,894,2,62.0,0,0,9.6875,0,1,0,1,0,0
3,895,3,27.0,0,0,8.6625,0,1,0,0,1,0
4,896,3,22.0,1,1,12.2875,1,0,0,0,1,0
5,897,3,14.0,0,0,9.2250,0,1,0,0,1,0
6,898,3,30.0,0,0,7.6292,1,0,0,1,0,1
7,899,2,26.0,1,1,29.0000,0,1,0,0,1,0
8,900,3,18.0,0,0,7.2292,1,0,1,0,0,1
9,901,3,21.0,2,0,24.1500,0,1,0,0,1,0


In [ ]:
predictions = df_test[['PassengerId','Survived']]

In [ ]:
predictions.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [ ]:
#exporting the predictions
predictions.to_csv('final_predictions.csv', index=False)